In [155]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data

In [ ]:
device = torch.device('cpu') # torch.device('cuda')

## Data

In [156]:
artists = pd.read_csv("../../../data/processed/artists.csv")
tracks = pd.read_json("../../../data/v2/tracks.json")
users = pd.read_json("../../../data/v2/users.json")
sessions = pd.read_json("../../../data/v2/sessions.json")

In [157]:
track_ids = tracks.id
VALID_COLUMN_NAMES = ['id', 'duration_ms', 'popularity', 'explicit', 'release_date','danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
tracks = tracks[VALID_COLUMN_NAMES]
rd = tracks.release_date
rd = pd.to_datetime(rd, errors='coerce')
tracks['release_date'] = rd.dt.year.fillna(0).astype(int)

In [158]:
users = users.drop(columns=['name', 'city', 'street'])
GENRES = np.unique(np.concatenate(users['favourite_genres'].to_numpy()))

In [159]:
to_drop = sessions['event_type'] == 'skip'
to_drop[len(to_drop)] = False
to_drop = [ (to_drop[i+1] or to_drop[i]) for i in range(len(to_drop) - 1) ]
to_drop = np.array(to_drop)
to_drop += sessions['event_type'] == 'advertisment'
to_drop = ~to_drop
sessions = sessions[to_drop]

In [160]:
sessions.head()

,session_id,timestamp,user_id,track_id,event_type
0,124,2021-11-18 10:40:38.000,101,7G67ZJRQT9nn2Fa9vA6B32,play
2,124,2021-11-18 10:41:48.495,101,7G67ZJRQT9nn2Fa9vA6B32,like
4,124,2021-11-18 10:46:00.240,101,6xagjcywpcyNFghafZPQJv,play
8,124,2021-11-18 10:47:54.348,101,70CYAL35X3T73qVStJNpZ2,play
9,124,2021-11-18 10:50:13.137,101,70CYAL35X3T73qVStJNpZ2,like


In [161]:
users.head()
users = users[:10]

In [162]:
tracks.head()

,id,duration_ms,popularity,explicit,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0RNxWy0PC3AyH4ThH3aGK6,201467,55,0,1929,0.673,0.3770,0,-14.141,0.0697,0.586,0.000000,0.332,0.713,88.973
1,2W889aLIKxULEefrleFBFI,198000,54,0,1943,0.204,0.1510,2,-17.842,0.0418,0.947,0.000009,0.321,0.134,91.783
2,4Pnzw1nLOpDNV6MKI5ueIR,199000,55,0,1944,0.295,0.0826,1,-19.569,0.0367,0.984,0.000358,0.156,0.169,128.600
3,7GLmfKOe5BfOXk7334DoKt,163000,54,0,1944,0.561,0.3350,9,-11.093,0.0499,0.840,0.000002,0.788,0.590,126.974
4,6kD1SNGPkfX9LwaGd1FG92,186173,53,0,1944,0.197,0.0546,1,-22.411,0.0346,0.950,0.276000,0.152,0.100,90.150


## Model

Classifier input: <br>
    1. Track Desccription (12 numerical + 2 categorical giving 16 one-hots + 1 binary) <br>
    2. User Description (2 categorical giving 50 one-hots + 1 binary) <br>
    3. Music Fingerprint (5 categorical giving 5x50 one-hots) <br>
Classifier output: <br>
    1. P(User will listen whole track) <br>
    2. P(User will like the track) <br>
    3. P(Track is similar to fingerprint) <br>

In [164]:
u = users.loc[1]
t = tracks.loc[640] # 640 play // 8428 like
select = sessions[u['user_id'] == sessions['user_id']]
select = select[t['id'] == select['track_id']]
play = bool(sum(select['event_type'] == 'play'))
like = bool(sum(select['event_type'] == 'like'))
ug = u['favourite_genres']
ug = torch.Tensor(np.array([i in ug for i in GENRES]) * 1)
ur = torch.Tensor([u['premium_user'] * 1])
tk = torch.Tensor([int(i == t['key']) for i in range(16)])
tr = torch.Tensor(t.drop(labels=['id', 'key']).to_numpy().astype(np.float64))
# [[ug, ur], [tk, tr]], [play, like]

In [165]:
tracks[tracks['id'] == '0Mn3amMRMoabaoTf1Publ4']

,id,duration_ms,popularity,explicit,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
8428,0Mn3amMRMoabaoTf1Publ4,205746,70,1,2015,0.594,0.749,3,-6.251,0.0677,0.00188,0.000139,0.0869,0.344,130.064


In [166]:
class MusicDataset(data.Dataset):
    def __init__(self, users, tracks, sessions):
        self.users = users
        self.lusers = len(users)
        self.tracks = tracks
        self.ltracks = len(tracks)
        self.sessions = sessions

    def __len__(self):
        return self.ltracks * self.lusers

    def __getitem__(self, idx):
        u = users.loc[idx // self.ltracks]
        t = tracks.loc[idx % self.ltracks]
        select = sessions[u['user_id'] == sessions['user_id']]
        select = select[t['id'] == select['track_id']]
        play = bool(sum(select['event_type'] == 'play'))
        like = bool(sum(select['event_type'] == 'like'))
        ug = u['favourite_genres']
        ug = torch.Tensor(np.array([i in ug for i in GENRES]) * 1)
        ur = torch.Tensor([u['premium_user'] * 1])
        tk = torch.Tensor([int(i == t['key']) for i in range(16)])
        tr = torch.Tensor(t.drop(labels=['id', 'key']).to_numpy().astype(np.float64))

        return [[ug, ur], [tk, tr]], [play, like]

In [167]:
class Music_classifier(nn.Module):
    def __init__(self, genres):
        self.genres = genres
        #    |
        # 50 | Genres | 10    }
        #    |                } USER [11]
        # czy_premium | 1     }
    
        #    |
        # 16 | key    | 3     }
        #    |                } TRACK [16]
        # numeric     | 12    }
        # binary      | 1     }
    
        # TRACK [16] |        }
        # TRACK [16] |   | 10 } FINGERPRINT [10]
        # TRACK [16] |        }
        track_key_code = 3
        user_genre_code = 10
        fingerprint_params = 15
        super(Music_classifier, self).__init__()
        ##### USER PREP ######
        self.emb_user_genre = nn.Linear(len(genres), user_genre_code)
        self.emb_user_genre_act = nn.LeakyReLU()

        ##### TRACK PREP #####
        self.emb_track_key = nn.Linear(16, track_key_code)
        self.emb_track_key_act = nn.LeakyReLU()

        ##### FINGERPRINT PREP #####
        # self.emb_fingerprint = nn.Linear(3 * (track_key_code + 12 + 1), fingerprint_params)
        # self.emb_fingerprint_act = nn.LeakyReLU()
        # self.emb_fingerprint_track1 = nn.Linear(16, track_key_code)
        # self.emb_fingerprint_track1_act = nn.LeakyReLU()
        # self.emb_fingerprint_track2 = nn.Linear(16, track_key_code)
        # self.emb_fingerprint_track2_act = nn.LeakyReLU()
        # self.emb_fingerprint_track3 = nn.Linear(16, track_key_code)
        # self.emb_fingerprint_track3_act = nn.LeakyReLU()

        #### MAIN CLASSIFIER ####
        user_params = user_genre_code + 1
        track_params = track_key_code + 12 + 1

        self.layers = nn.Sequential(
            # nn.Linear(user_params + track_params + fingerprint_params, 256),
            nn.Linear(user_params + track_params, 128),
            nn.LeakyReLU(),

            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.Dropout(0.1),
            nn.LeakyReLU(),

            nn.Linear(64, 32),
            nn.ReLU(),
            #nn.Linear(32, 3),
            nn.Linear(32, 2),

            nn.Sigmoid()
        )
    #def forward(self, user, track, finger):
    def forward(self, x):
        # user -> [Tensor(50, bs), Tensor(1, bs)]
        # track -> [Tensor(16, bs), Tensor(13, bs)]
        # finger -> [ [Tensor(16, bs), Tensor(13, bs)], [Tensor(16, bs), Tensor(13, bs)], [Tensor(16, bs), Tensor(13, bs)] ]
        user, track = x

        comp_emb_user_genre = self.emb_user_genre(user[0])
        comp_emb_user_genre = self.emb_user_genre_act(comp_emb_user_genre)

        comp_emb_track_key = self.emb_track_key(track[0])
        comp_emb_track_key = self.emb_track_key_act(comp_emb_track_key)

        # comp_emb_fingerprint_track1 = self.emb_fingerprint_track1(finger[0][0])
        # comp_emb_fingerprint_track1 = self.emb_fingerprint_track1_act(comp_emb_fingerprint_track1)
        # comp_emb_fingerprint_track2 = self.emb_fingerprint_track2(finger[1][0])
        # comp_emb_fingerprint_track2 = self.emb_fingerprint_track2_act(comp_emb_fingerprint_track2)
        # comp_emb_fingerprint_track3 = self.emb_fingerprint_track3(finger[2][0])
        # comp_emb_fingerprint_track3 = self.emb_fingerprint_track3_act(comp_emb_fingerprint_track3)
        # emb_finger_x = torch.cat([comp_emb_fingerprint_track1, finger[0][1], 
        #                           comp_emb_fingerprint_track2, finger[1][1], 
        #                           comp_emb_fingerprint_track3, finger[2][1]], dim=1)
        # comp_emb_fingerprint = self.emb_fingerprint(emb_finger_x)
        # comp_emb_fingerprint = self.emb_fingerprint_act(comp_emb_fingerprint)

        # x = torch.cat([comp_emb_user_genre, user[1], comp_emb_track_key, track[1], comp_emb_fingerprint])
        # print(comp_emb_user_genre.shape, user[1].shape, comp_emb_track_key.shape, track[1].shape)
        c = torch.cat([comp_emb_user_genre, user[1], comp_emb_track_key, track[1]], 1)
        
        return self.layers(c)


## Training

In [168]:
def accuracy(loader, model, device):
    with torch.no_grad():
        good_like, good_play, all = 0
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            preds = model(x)
            good_play = sum([a == b for a, b in zip(np.round(preds.detach().numpy(), 0).T[0], y[0])])
            good_like = sum([a == b for a, b in zip(np.round(preds.detach().numpy(), 0).T[1], y[1])])
            all = len(preds)
        print(f"Accuracy of playing: {good_play * 100/all:.3}%\nAccuracy of liking: {good_like * 100/all:.3}%\n")
           

In [169]:
dataset = MusicDataset(users, tracks, sessions)
train, test = data.random_split(dataset, [round(len(dataset) * 0.7), round(len(dataset) * 0.3)])

In [170]:
train_dataloader = data.DataLoader(train, batch_size=64, shuffle=True, drop_last=True)
test_dataloader = data.DataLoader(test, batch_size=64, shuffle=False, drop_last=False)

In [171]:
model = Music_classifier(GENRES)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_module = nn.CrossEntropyLoss()

In [172]:
EPOCHS = 1
model.train()
for epoch in range(EPOCHS):
    for x, y in train_dataloader:
        preds = model(x)
        loss = loss_module(preds, torch.cat([y[0].unsqueeze(1), y[1].unsqueeze(1)], 1).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}, loss: {loss.sum().item():.3}')
    if(epoch % 5):
        accuracy(train_dataloader, model, device)


Epoch: 0, loss: 0.155
